### 1. Dataset Collection and Preprocessing

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
from nltk.corpus import gutenberg
gutenberg.words()

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', ...]

In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string


# Choose a text from the Gutenberg corpus
text_id = 'shakespeare-macbeth.txt'
raw_text = gutenberg.raw(text_id)



In [29]:
files = gutenberg.fileids()
text = [gutenberg.raw(fileid) for fileid in gutenberg.fileids()]

file_text = dict(zip(files, text))

In [23]:
#cleaning and preprocessing data
#text cleaning - remove non-alphanumeric/special characters, removing stop words, lower case

def preprocess(text):
    #Tokenize lowercase, and remove stop words
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

#Bishal Agarwal - can avoid removing stopwords. 


In [27]:
#testing on one file-text pair
raw_text_pre = preprocess(raw_text)
print(raw_text[:50])
print(raw_text_pre[:5])

[The Tragedie of Macbeth by William Shakespeare 16
['tragedie', 'macbeth', 'william', 'shakespeare', 'actus']


In [37]:
#Run this for rhe entire file_text dictionary and generate tokens
for key, value in file_text.items():
    file_text[key] = preprocess(value)


In [223]:
file_text['shakespeare-macbeth.txt']

['tragedie',
 'macbeth',
 'william',
 'shakespeare',
 'actus',
 'primus',
 'scoena',
 'prima',
 'thunder',
 'lightning',
 'enter',
 'three',
 'witch',
 'shall',
 'three',
 'meet',
 'againe',
 'thunder',
 'lightning',
 'raine',
 'done',
 'battaile',
 'lost',
 'wonne',
 'ere',
 'set',
 'sunne',
 'place',
 'vpon',
 'heath',
 'meet',
 'macbeth',
 'come',
 'padock',
 'call',
 'anon',
 'faire',
 'foule',
 'foule',
 'faire',
 'houer',
 'fogge',
 'filthie',
 'ayre',
 'exeunt',
 'scena',
 'secunda',
 'alarum',
 'within',
 'enter',
 'king',
 'malcome',
 'donalbaine',
 'lenox',
 'attendant',
 'meeting',
 'bleeding',
 'captaine',
 'king',
 'bloody',
 'man',
 'report',
 'seemeth',
 'plight',
 'reuolt',
 'newest',
 'state',
 'mal',
 'serieant',
 'like',
 'good',
 'hardie',
 'souldier',
 'fought',
 'captiuitie',
 'haile',
 'braue',
 'friend',
 'say',
 'king',
 'knowledge',
 'broyle',
 'thou',
 'didst',
 'leaue',
 'cap',
 'doubtfull',
 'stood',
 'two',
 'spent',
 'swimmer',
 'doe',
 'cling',
 'togethe

## 2. Model Implementation

In [165]:
#define n
n = 3

In [174]:
#creating n-grams from tokenised text
#creating unigram, bigrams, trigtams from tokens

#can also use ntlk.ngrams()

def create_ngrams(tokens, n):
    n_gram_tokens = []
    for i in range(len(tokens)-n):
        n_gram_tokens.append(tuple(tokens[i:i+n]))
    return n_gram_tokens

In [175]:
#second part - calcualte frequency
sample = file_text['austen-emma.txt']
ngrams_sample = create_ngrams(sample,n)
ngrams_sample[:6]

[('emma', 'jane', 'austen'),
 ('jane', 'austen', 'volume'),
 ('austen', 'volume', 'chapter'),
 ('volume', 'chapter', 'emma'),
 ('chapter', 'emma', 'woodhouse'),
 ('emma', 'woodhouse', 'handsome')]

In [96]:
from collections import Counter
ngram_frequency = Counter([tuple(ngram) for ngram in ngrams_sample])

In [205]:
#calculating the probability of a word following a given n-1 gram

def probability_helper(sample,n):
    """
    sample: text sample
    n: n-gram size
    return: dataframe with probability
    """
    #get ngrams
    ngrams_sample = create_ngrams(sample,n)

    #get frequency
    ngram_frequency = Counter([tuple(ngram) for ngram in ngrams_sample])

    #ger probability
    df = pd.DataFrame.from_dict(ngram_frequency, orient='index').reset_index()
    df.columns = ['sequence',  'count']

    #convert first column into 2 columns where first column has n-1 words, the second column has nth word
    df['nth_word'] = df['sequence'].apply(lambda x: x[-1])

    def get_sequence(tuple):
        x = ''
        for i in range(len(tuple)-1):
            x+=(tuple[i])
            x+=','
        x = x[:-1]
        x = x.replace(","," ")
        return x

    df['sequence'] = df['sequence'].apply(lambda x: get_sequence(x))

    #get ids for sequences and predictions
    df_sorted = df.sort_values(by='sequence')
    df_sorted['sequence_id'] = range(1, len(df_sorted) + 1)
    df_new = df_sorted
    df_sorted = df_new.sort_values(by='nth_word')
    df_sorted['prediction_id'] = range(1, len(df_sorted) + 1)

    return df, df_sorted

def get_probability(sample,n,type = None):
    if type==None:

        df, df_sorted = probability_helper(sample,n)
        totals = df.groupby('sequence')['count'].sum().reset_index().rename(columns={'count':'total'})
        df_sorted = df_sorted.merge(totals, how = 'left', on = 'sequence')
        df_sorted['probability'] = df_sorted['count']/df_sorted['total']
    elif type =="smooth":
        df, df_sorted = probability_helper(sample,n)
        v = df_sorted['prediction_id'].max()
        
        totals = df.groupby('sequence')['count'].sum().reset_index().rename(columns={'count':'total'})
        df_sorted = df_sorted.merge(totals, how = 'left', on = 'sequence')
        df_sorted['probability'] = (df_sorted['count']+1)/(df_sorted['total'] + v)

    return df_sorted



In [206]:
probs  = get_probability(sample, n)
probs

,sequence,count,nth_word,sequence_id,prediction_id,total,probability
0,hartfield donwell,1,abbey,26150,1,1,1.000000
1,secondary donwell,1,abbey,51925,2,1,1.000000
2,owner donwell,1,abbey,43409,3,1,1.000000
3,sheltered rose,1,abbey,53402,4,1,1.000000
4,belonging haunting,1,abbey,5026,5,1,1.000000
...,...,...,...,...,...,...,...
68560,drawing much,1,zeal,14600,68561,1,1.000000
68561,watched safely,1,zeal,63949,68562,1,1.000000
68562,idea greatest,1,zeal,28227,68563,1,1.000000
68563,shall try,1,zeal,53327,68564,3,0.333333


In [224]:
probs[probs['sequence']=='shall try']
#incorporate randomness when probability is equal

,sequence,count,nth_word,sequence_id,prediction_id,total,probability
25852,shall try,1,harriet,53328,25853,3,0.333333
44906,shall try,1,persuade,53329,44907,3,0.333333
68563,shall try,1,zeal,53327,68564,3,0.333333


In [153]:
#df_sorted.pivot(index = ['sequence_id'], columns = ['prediction_id'], values = 'count')
#creating an array was too big
#so we will get totals and get a probaility by a simple divide

In [344]:
import random
# Write a function to predict the next word given the sequence of words
def predict(data, sequence):
    """this function generates predictions based on probabilities seen in the dataset"""
    try:
        subset = data[data['sequence']==sequence.strip()]
        result = subset.iloc[subset['probability'].argmax()]['nth_word'] #return the word with max probability
        #print("sequence detected")
        return result
    except:
        result = random.choice(data['nth_word'].unique())
        #print("sequence not detected")
        return result
    #test

print(predict(probs,'secondary donwell'))

abbey


In [341]:
#Write a function to generate a sentence of a specified length given a prefix of (n−1) words

def generate_sentence(data, sequence, n,len ):
    """
    data: result of get_probability()
    sequence: should be n-1 words together
    len: number of predictions to be made
    """
    sentence = sequence
    sentence = sentence.strip()
    for i in range(len):
        n_minus_1_sequence = ' '.join(sentence.split(" ")[-n+1:])
        print(f'sequence number {i+1}: {n_minus_1_sequence}')
        next_word = predict(data, n_minus_1_sequence)
        sentence = sentence + ' ' + next_word
    return sentence
    
result = generate_sentence(probs, "secondary donwell", 3,5)
print(result)

sequence number 1: secondary donwell
sequence detected
sequence number 2: donwell abbey
sequence detected
sequence number 3: abbey chapter
sequence detected
sequence number 4: chapter vi
sequence detected
sequence number 5: vi emma
sequence detected
secondary donwell abbey chapter vi emma could


In [315]:
#Implement smoothing techniques (like Laplace smoothing) to handle the issues of zero probabilities for unseen n-grams.
#Updating the probability function to incorporate laplac smoothing
def get_probability(sample,n,type = None):
    if type==None:
        df, df_sorted = probability_helper(sample,n)
        totals = df.groupby('sequence')['count'].sum().reset_index().rename(columns={'count':'total'})
        df_sorted = df_sorted.merge(totals, how = 'left', on = 'sequence')
        df_sorted['probability'] = df_sorted['count']/df_sorted['total']

    elif type =="smooth":
        df, df_sorted = probability_helper(sample,n)
        v = df_sorted['prediction_id'].max()
        totals = df.groupby('sequence')['count'].sum().reset_index().rename(columns={'count':'total'})
        df_sorted = df_sorted.merge(totals, how = 'left', on = 'sequence')
        df_sorted['probability'] = (df_sorted['count']+1)/(df_sorted['total'] + v)

    return df_sorted

In [220]:
#to show a demonstration of this, we need to get probabilities for laplace smoothing and send in a sequence that is not existing in the dataset
probs_smooth = get_probability(sample, n,type = "smooth")
result = generate_sentence(probs_smooth, "secondary something", 5)
print(result)

does not exist
does not exist
does not exist
does not exist
does not exist
secondary something sensibility educate insufficient received among


This shows that for n-grams when n is 3:

the first sequence: "secondary something" does not exist in the dataset, 

second sequence: "something sensibility" does not exist either and 

similarly, "sensibility educate", "educate insufficient"... dont exist in the dataset and yet we are able to generate predictions using Laplace Smoothing in the PDF.

## 3. Testing and Evaluation

In [221]:
##3 Test the model with different (n − 1)-gram inputs to see how it predicts the next word.

In [272]:
import string
def preprocess_new(text):
    
    #Tokenize lowercase, and remove stop words
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    # Remove punctuation from the list of tokens
    tokens = [word for word in tokens if word not in string.punctuation]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens
    
files = gutenberg.fileids()
text = [gutenberg.raw(fileid) for fileid in gutenberg.fileids()]
file_text = dict(zip(files, text))

for key, value in file_text.items():
    file_text[key] = preprocess_new(value)

In [348]:
n = 2
test_file = file_text['blake-poems.txt']
probs_test  = get_probability(test_file, n,type = "smooth")
result = generate_sentence(probs_test, "what  ",n, 5)
print(result)

sequence number 1: what
sequence number 2: caterpillar
sequence number 3: fly
sequence number 4: feed
sequence number 5: little
what caterpillar fly feed little boy


In [349]:
n = 2
test_file = file_text['blake-poems.txt']
probs_test  = get_probability(test_file, n,type = "smooth")
result = generate_sentence(probs_test, "caterpillar  ",n, 5)
print(result)

sequence number 1: caterpillar
sequence number 2: fly
sequence number 3: feed
sequence number 4: little
sequence number 5: boy
caterpillar fly feed little boy lost


In [346]:
n = 3
test_file = file_text['blake-poems.txt']
probs_test  = get_probability(test_file, n,type = "smooth")
result = generate_sentence(probs_test, "blight plague",n, 5)
print(result)

sequence number 1: blight plague
sequence number 2: plague human
sequence number 3: human abstract
sequence number 4: abstract pity
sequence number 5: pity would
blight plague human abstract pity would make


In [345]:
n = 4
test_file = file_text['blake-poems.txt']
probs_test  = get_probability(test_file, n,type = "smooth")
result = generate_sentence(probs_test, "blight plague human ",n, 5)
print(result)

sequence number 1: blight plague human
sequence number 2: plague human abstract
sequence number 3: human abstract pity
sequence number 4: abstract pity would
sequence number 5: pity would make
blight plague human abstract pity would make somebody


In [347]:
n = 5
test_file = file_text['blake-poems.txt']
probs_test  = get_probability(test_file, n,type = "smooth")
result = generate_sentence(probs_test, "blight plague human is ",n, 5)
print(result)

sequence number 1: blight plague human is
sequence number 2: plague human is shriek
sequence number 3: human is shriek nostril
sequence number 4: is shriek nostril fallen
sequence number 5: shriek nostril fallen frame
blight plague human is shriek nostril fallen frame end


2

In [350]:
#Compare the performance of different n values (e.g., bigrams vs. trigrams)

probs_test  = get_probability(test_file, n,type = "smooth")
probs_test['log_probability'] = np.log(probs_test['probability'])
# Sum of log probabilities
sum_log_prob = probs_test['log_probability'].sum()
# Number of predictions
N = len(probs_test)
# Calculate perplexity
perplexity = np.exp(-sum_log_prob / N)

print("Perplexity:", perplexity)

Perplexity: 1675.4673943165446


In [352]:
n = 3
probs_test  = get_probability(test_file, n,type = "smooth")
probs_test['log_probability'] = np.log(probs_test['probability'])
# Sum of log probabilities
sum_log_prob = probs_test['log_probability'].sum()
# Number of predictions
N = len(probs_test)
# Calculate perplexity
perplexity = np.exp(-sum_log_prob / N)

print("Perplexity:", perplexity)

Perplexity: 1813.0033448299969


In [353]:
n = 4
probs_test  = get_probability(test_file, n,type = "smooth")
probs_test['log_probability'] = np.log(probs_test['probability'])
# Sum of log probabilities
sum_log_prob = probs_test['log_probability'].sum()
# Number of predictions
N = len(probs_test)
# Calculate perplexity
perplexity = np.exp(-sum_log_prob / N)

print("Perplexity:", perplexity)

Perplexity: 1842.8593390221017


In [354]:
n = 5
probs_test  = get_probability(test_file, n,type = "smooth")
probs_test['log_probability'] = np.log(probs_test['probability'])
# Sum of log probabilities
sum_log_prob = probs_test['log_probability'].sum()
# Number of predictions
N = len(probs_test)
# Calculate perplexity
perplexity = np.exp(-sum_log_prob / N)

print("Perplexity:", perplexity)

Perplexity: 1852.0622740820113


Perplexity is a measure of uncertainty or "surprise" of a language model. A lower perplexity indicates that the language model is more confident in its predictions (i.e., less surprised by the test data). This typically means the model is more accurate in predicting the probability of a sequence of words.

## 4. Short Report

In this short notebook, we defined various functions that helped us achieve tasks necessary to build an n-gram model i.e.
1. preprocess text 
2. derive n-grams
3. calculate frequency/count
4. calculate probability to define PDFs
5. Use PDFs to predict words

In the end we run examples to predict sentences from an initial sequence for different n-gram models. We saw that as n increases, the likelihood of the model, encountering a 'never-seen-before' sequence also increases. In other words, the model can be more 'perplexed' at the sequence it sees.

#### Perplexity in n-gram models:
Increasing the 'n' in an n-gram model generally makes the model more powerful because it can capture longer dependencies. However, this does not always lead to lower perplexity. The impact on perplexity can depend on various factors, such as the amount and representativeness of the training data, and the handling of unseen n-grams.

With a higher 'n', the model becomes more specific, potentially leading to better performance on similar types of data as seen during training. However, it might also lead to increased sparsity and overfitting, especially if the training data is not sufficiently large and diverse.

The model might become too sensitive to the specific training data and fail to generalize well to new, unseen data, which can actually increase perplexity. This is particularly true for rare n-grams that do not appear often in the training data.

In summary, while a more complex model (with a higher 'n' in an n-gram model) might theoretically be more powerful, in practice, its effectiveness and impact on perplexity depend on factors like data availability, overfitting, and how well it can generalize beyond the training data.





